This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/clutter.html).  I recommend having both windows open, side-by-side!

In [ ]:
import numpy as np
from pydrake.all import Rgba, StartMeshcat
from pydrake.multibody.benchmarks import MassDamperSpringAnalyticalSolution

from manipulation import running_as_notebook

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

# Let's explore stiffness

As "spring-mass-damper" system is enough to make the basic point.

In [ ]:
def spring_mass_damper_example():
    meshcat.Delete()
    meshcat.DeleteAddedControls()
    meshcat.SetProperty("/Background", "visible", False)
    meshcat.Set2dRenderMode(xmin=-2, xmax=2, ymin=-2, ymax=2)

    meshcat.AddSlider(name="mass", min=0.02, max=10, value=1.0, step=0.1)
    meshcat.AddSlider(name="b", min=0.02, max=10, value=1.0, step=0.1)
    meshcat.AddSlider(
        name="k",
        min=0.02,
        max=10,
        value=1.0,
        step=0.1,
        decrement_keycode="ArrowLeft",
        increment_keycode="ArrowRight",
    )
    meshcat.AddSlider(
        name="h",
        min=0.02,
        max=10,
        value=0.1,
        step=0.1,
        decrement_keycode="ArrowDown",
        increment_keycode="ArrowUp",
    )

    meshcat.AddButton("Stop Demo", "Escape")

    N = 200
    tf = 20
    ts = np.linspace(0, tf, N)
    xs = np.zeros((N, 2))
    Xs = np.zeros((3, N))

    Nd = 200
    xd = np.zeros((Nd, 2))
    Xd = np.zeros((3, Nd))

    prev = None

    while meshcat.GetButtonClicks("Stop Demo") < 1:
        mass = meshcat.GetSliderValue("mass")
        b = meshcat.GetSliderValue("b")
        k = meshcat.GetSliderValue("k")
        h = meshcat.GetSliderValue("h")
        if [mass, b, k, h] == prev:
            continue
        prev = [mass, b, k, h]

        sol = MassDamperSpringAnalyticalSolution(mass, b, k)
        sol.SetInitialValue(1, 0)

        for i in range(N):
            xs[i] = [sol.get_x(ts[i]), sol.get_xDt(ts[i])]
        Xs[0, :] = xs[:, 0]
        Xs[2, :] = xs[:, 1]
        meshcat.SetLine("analytical", Xs, line_width=4, rgba=Rgba(0, 0, 1, 1))

        xd[0] = [1, 0]
        Ad = np.eye(2) + h * np.array([[0, h], [-k * h / mass, -b * h / mass]])
        for i in range(1, Nd):
            xd[i] = Ad @ xd[i - 1]
        Xd[0, :] = xd[:, 0]
        Xd[2, :] = xd[:, 1]
        meshcat.SetLine("euler", Xd, line_width=4, rgba=Rgba(1, 0, 0, 1))

        if not running_as_notebook:
            break

    meshcat.DeleteButton("Stop Demo")


spring_mass_damper_example()